# Scraping Tutorial

This Scraping Tutorial created for Tracking Injustice to aid in the creation of a living dataset tracking Canadian Police-Involved Deaths. For more information regarding Tracking Injustice, see https://trackinginjustice.ca/.

This tutorial is run and created by Emily Medema and Rohan Khan.

## Interactive Scraping Example

The best way to learn how to scrape is to do it. As we have seen in the slides (which can be found here: [Slide Link]("http://tiny.cc/scraping-tutorial-slides")), websites differ greatly and therefore our scraping techniques have to be customized to the site. The best way to learn to do that, while following the guidelines also laid out in the slides, is to do it yourself. Scraping is also a continuous process of creating and maintainence. A website may change which can cause your script to become ineffective. You must be able to adapt so that your script can continue to be effective.

First, we will get our environment setup to scrape and then we will work our way up from simply examples to more and more complex ones. 

### Environmental Setup

We are using Jupyter Notebooks, which is an open source web application that you can use to create and share documents that contain live code, equations, visualizations, and text. It is incredibly useful for Python. Through jupyter notebooks you can create a document that documents, explains, and contains your code all in one place. This is very helpful for maintainence purposes.

Jupyter Notebooks run from either third-party sites, your virtual environment, or your localhost. Therefore, you can access whatever python libaries installed either on that third-party site, your virtual environment, or your own machine by simply importing the libary. If you ever have to install a libary on the notebook itself you can do so with this command:

```
!pip install libaryname
```

For this scraping tutorial, we will want the following libraries:

- pandas
- numpy
- urllib3
- bs4
- MechanicalSoup
- Scrapy
- selenium

You will most likely never need all of these for scraping. Nevertheless, ensure that all these are installed on whatever machine you are running jupyter notebook on.

You can easily install all of these libraries if you have the [github repository]("https://github.com/emedema/scraping_tutorial") cloned by running the following command:

```
pip install -r requirements.txt
```

Now that we have all these libraries installed, we can import them into the notebook. This means we can then use their methods etc. within our code.

In [2]:
# import libraries
import pandas as pd
import numpy as np
import urllib3
from bs4 import BeautifulSoup
import mechanicalsoup
import scrapy
import selenium